In [2]:
import string

In [3]:
# onp functions

In [4]:
def getVariables(expression):
    variables = [letter for letter in expression if letter.isalpha()]
    return ''.join(sorted(set(variables)))

In [5]:
def validate(expression):
    variables = string.ascii_lowercase + 'T' + 'F'
    operators = "&>|/>"
    state = True
    counter = 0

    for char in expression:
        if char == '(': counter += 1
        if char == ')': counter -= 1
        if counter < 0: return False
        if state:
            if char == '~': state = True
            elif char in variables: state = False
            elif char in ')' + operators: return False
        else:
            if char in operators: state = True
            elif char in '(' + variables + '~': return False
    if counter != 0: return False
    return not state

In [6]:
def divideExpression(expression, operators):
    counter = 0
    for i in range(len(expression) - 1, -1, -1):       # zmieniłem 0 na -1
        if expression[i] == ')': counter += 1
        if expression[i] == '(': counter -= 1
        if expression[i] in operators and counter == 0: return i
    return -1

In [7]:
def onp(expression):
    operators = "&>|/"
    while expression[0] == '(' and expression[-1] == ')' and validate(expression[1:-1]):
        expression = expression[1:-1]

    for p in [result for result in [divideExpression(expression, operator) for operator in ">|&/"] if result >= 0] :
        return onp(expression[:p]) + onp(expression[p + 1:]) + expression[p]
        
    p = divideExpression(expression, '~')
    if p >= 0:
        return onp(expression[p + 1:]) + expression[p]
        
    return expression

In [8]:
def substitude(expression, variables, values):
    mappedExpr = list(expression)
    for i in range(len(expression)):
        pos = variables.find(expression[i])
        if pos >= 0: mappedExpr[i] = values[pos]
    return ''.join(mappedExpr)

In [9]:
def evaluateLogic(expression, values):
    variables = getVariables(expression)
    #print(variables)
    mappedExpr = substitude(onp(expression), variables, values)
    #print(mappedExpr)
    stack = list("")
    for x in mappedExpr:
        if x in '01': stack.append(int(x))
        else:
            y1 = not not stack.pop()
            if x == '~': 
                stack.append(not y1)
            else: 
                y2 = not not stack.pop()
                if x == '&': stack.append(y1 and y2)
                elif x == '|': stack.append(y1 or y2)
                elif x == '>': stack.append(y1 or (not y2))
    return stack.pop()

In [10]:
def binaryGenerator(n):
    if n == 0: yield ""
    else:
        for c in binaryGenerator(n-1):
            yield "0" + c
            yield "1" + c

In [11]:
# Quine Mccluskey algorithm

In [12]:
def latch(vec1, vec2):
    diffCounter = 0
    result = ""
    for c1, c2 in zip(vec1, vec2):
        if c1 == c2:
            result += c1
        else:
            diffCounter += 1
            result += "-"
    if diffCounter == 1: return result
    else: return False

In [13]:
def readData():
    data = open("data.txt", "r")
    result = set(data.read().splitlines())
    data.close()
    return result

In [14]:
def quineMcCluskey(binaries):
    anyLatch = False
    result = set()
    for binary1 in binaries:
        foundedLatch = False
        for binary2 in binaries:
            vec = latch(binary1, binary2)
            if vec:
                result.add(vec)
                foundedLatch = anyLatch = True
        if not foundedLatch: result.add(binary1)
    if anyLatch: return quineMcCluskey(result)
    return result

In [15]:
def convertToLogic(expression):
    mainResult = ""
    for vec in expression:
        result = ""
        for elem in range(len(vec)):
            if vec[elem] == "-": continue
            if vec[elem] == "0": result += "~"
            result += string.ascii_lowercase[elem]+"&"
        mainResult += "(" + result[:-1] + ")|"
    return mainResult[:-1]


In [16]:
def check(expr, binaries):
    for result in [evaluateLogic(expr, binary) for binary in binaries]: print(result)

In [17]:
main()

NameError: name 'main' is not defined

In [18]:
    def unate_cover(self, primes, binaries):
        """
    Use the prime implicants to find the essential prime implicants of the
    function, as well as other prime implicants that are necessary to cover
    the function. This method uses the Petrick's method, which is a technique
    for determining all minimum sum-of-products solutions from a prime implicant
    chart.

    primes: the prime implicants that we want to minimize.
    binaries: a list of indices for the minterms for which we want the function to
    evaluate to 1.
    """

        chart = []
        for binary in binaries:
            column = []
            for i in range(len(primes)):
                if (binary & (~primes[i][1])) == primes[i][0]:
                    column.append(i)
            chart.append(column)

        covers = []
        if len(chart) > 0:
            covers = [set([i]) for i in chart[0]]
        for i in range(1, len(chart)):
            new_covers = []
            for cover in covers:
                for prime_index in chart[i]:
                    x = set(cover)
                    x.add(prime_index)
                    append = True
                    for j in range(len(new_covers) - 1, -1, -1):
                        if x <= new_covers[j]:
                            del new_covers[j]
                        elif x > new_covers[j]:
                            append = False
                    if append:
                        new_covers.append(x)
            covers = new_covers

        min_complexity = 99999999
        for cover in covers:
            primes_in_cover = [primes[prime_index] for prime_index in cover]
            complexity = self.calculate_complexity(primes_in_cover)
            if complexity < min_complexity:
                min_complexity = complexity
                result = primes_in_cover

        return min_complexity, result

    def calculate_complexity(self, minterms):
        """
    Calculate the complexity of the given function. The complexity is calculated
    based on the following rules:
    A NOT gate adds 1 to the complexity.
    A n-input AND or OR gate adds n to the complexity.

    minterms: a list of minterms that form the function

    returns: an integer that is the complexity of the function

    >>> qm = QM(['A','B','C'])

    >>> qm.calculate_complexity([(1,6)])
    0
    >>> qm.calculate_complexity([(0,6)])
    1
    >>> qm.calculate_complexity([(3,4)])
    2
    >>> qm.calculate_complexity([(7,0)])
    3
    >>> qm.calculate_complexity([(1,6),(2,5),(4,3)])
    3
    >>> qm.calculate_complexity([(0,6),(2,5),(4,3)])
    4
    >>> qm.calculate_complexity([(0,6),(0,5),(4,3)])
    5
    >>> qm.calculate_complexity([(0,6),(0,5),(0,3)])
    6
    >>> qm.calculate_complexity([(3,4),(7,0),(5,2)])
    10
    >>> qm.calculate_complexity([(1,4),(7,0),(5,2)])
    11
    >>> qm.calculate_complexity([(2,4),(7,0),(5,2)])
    11
    >>> qm.calculate_complexity([(0,4),(7,0),(5,2)])
    12
    >>> qm.calculate_complexity([(0,4),(0,0),(5,2)])
    15
    >>> qm.calculate_complexity([(0,4),(0,0),(0,2)])
    17
    """

        complexity = len(minterms)
        if complexity == 1:
            complexity = 0
        mask = (1 << self.numvars) - 1
        for minterm in minterms:
            masked = ~minterm[1] & mask
            term_complexity = bitcount(masked)
            if term_complexity == 1:
                term_complexity = 0
            complexity += term_complexity
            complexity += bitcount(~minterm[0] & masked)

        return complexity

In [22]:
main()

{'1100', '0111', '0100', '1010', '1101', '1111', '1001', '1110'}
{'-111', '1-10', '-100', '1-01', '11--'}


In [27]:
def constructPetricChart(primes):
    chart = []
    varCount = len(primes.pop())
    for i in range(varCount): chart.append([])
    for prime in primes:
        

In [28]:
def main():
    binaries = readData()
    result = quineMcCluskey(binaries)
    expr = convertToLogic(result)
    
    print(binaries)
    print(result)
    constructPetricChart(result)

#     print(expr)
#     print(validate(expr))
#     print(onp(expr))
#     print(evaluateLogic(expr, "0000"))
#     check(expr, binaries)
    

In [29]:
main()

{'1100', '0111', '0100', '1010', '1101', '1111', '1001', '1110'}
{'-111', '1-10', '-100', '1-01', '11--'}
4
